In [66]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/dnd_chars_all.tsv", sep="\t")
df['HP'] / df['level']

0        10.428571
1         9.500000
2        10.769231
3         6.800000
4         6.800000
           ...    
10889     8.333333
10890    10.666667
10891     9.666667
10892    10.000000
10893     8.500000
Length: 10894, dtype: float64

In [74]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/dnd_chars_all.tsv", sep="\t")
df.drop(columns=["ip", "finger", "hash"], inplace=True)
df["normalized_race"] = df["processedRace"].str.strip().str.lower()
df["normalized_background"] = df["background"].str.strip().str.lower()
print(df['AC'])
counts = df['justClass'].value_counts()
X = df[['Str', 'Dex', 'Con', 'Int', 'Wis', 'Cha', 'HP', 'AC', 'normalized_race', 'normalized_background', 'processedAlignment', 'level']]
y = df['justClass']

# Fill-in processedAlignment with Random alignments, maintaining original distribution
alignment_dist = X['processedAlignment'].value_counts(normalize=True)
na_mask = X['processedAlignment'].isna()
X.loc[na_mask, 'processedAlignment'] = np.random.choice(
    alignment_dist.index,
    size=na_mask.sum(),
    p=alignment_dist.values
)

# Remove classes that don't have enough samples
min_samples = 50
frequent_classes = counts[counts >= min_samples].index
mask = y.isin(frequent_classes)
X = X[mask]
X['StrCon'] = X['Str'] - X['Con']
X['WisInt'] = X['Wis'] - X['Int']
X['HPPerLevel'] = X['HP']/X['level']
X.drop(['level'], axis=1, inplace=True)
y = y[mask]
original_counts = df['justClass'].value_counts()
max_count = original_counts.max()
min_count = original_counts.min()
original_imbalance_ratio = max_count / min_count
print(f"Original Imbalance ratio: {original_imbalance_ratio:.2f} (1.0 = perfectly balanced)")
pruned_counts = y.value_counts()
max_count = pruned_counts.max()
min_count = pruned_counts.min()
pruned_imbalance_ratio = max_count / min_count
print(f"Pruned Imbalance ratio: {pruned_imbalance_ratio:.2f} (1.0 = perfectly balanced)")

0        10
1        10
2        21
3        16
4        16
         ..
10889    13
10890    16
10891    19
10892    14
10893    13
Name: AC, Length: 10894, dtype: int64
Original Imbalance ratio: 1342.00 (1.0 = perfectly balanced)
Pruned Imbalance ratio: 6.58 (1.0 = perfectly balanced)


In [75]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numerical_cols = ['Str', 'Dex', 'Con', 'Int', 'Wis', 'Cha', 'HP', 'AC', 'StrCon', 'WisInt', 'HPPerLevel']
categorical_cols = ['normalized_race', 'normalized_background',
            'processedAlignment']

preprocess = ColumnTransformer([
        ("num", StandardScaler(), numerical_cols),
        ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.dummy import DummyClassifier
import numpy as np


dummy_pipe = Pipeline([
        ("prep",  preprocess),
        ("model", DummyClassifier(strategy='most_frequent'))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_dummy = cross_val_score(dummy_pipe, X, y, cv=cv, scoring='f1_macro')
print("5-fold dummy macro-F1:", cv_dummy.mean())

pipe = Pipeline([
        ("prep",  preprocess),
        ("model", OneVsRestClassifier(LogisticRegression(max_iter=1000)))
])
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipe, X, y,
                         cv=cv,
                         scoring="f1_macro")
print("5-fold macro-F1:", scores.mean())

In [ ]:
from sklearn.calibration import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

number_of_estimators = 100

unbalanced_pipe = Pipeline([
        ("prep",  preprocess),
        ("model", RandomForestClassifier(
            n_estimators=number_of_estimators, 
            random_state=42, 
        ))
])

balanced_pipe = Pipeline([
        ("prep",  preprocess),
        ("model", RandomForestClassifier(
            n_estimators=number_of_estimators, 
            random_state=42,
            class_weight='balanced'
        ))
])

balanced_subsample_pipe = Pipeline([
        ("prep",  preprocess),
        ("model", RandomForestClassifier(
            n_estimators=number_of_estimators, 
            random_state=42,
            class_weight='balanced_subsample'
        ))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(unbalanced_pipe, X, y,
                         cv=cv,
                         scoring="f1_macro")
predictions = cross_val_predict(unbalanced_pipe, X, y, cv=cv)
print("Unbalanced f1:", scores.mean())
print(classification_report(y, predictions))


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(balanced_pipe, X, y,
                         cv=cv,
                         scoring="f1_macro")
predictions = cross_val_predict(balanced_pipe, X, y, cv=cv)

print("Balanced f1:", scores.mean())
print(classification_report(y, predictions))


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(balanced_subsample_pipe, X, y,
                         cv=cv,
                         scoring="f1_macro")
predictions = cross_val_predict(balanced_subsample_pipe, X, y, cv=cv)
print("Balanced subsample f1:", scores.mean())
print(classification_report(y, predictions))

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
scores = cross_val_score(balanced_pipe, X, y, cv=rskf, scoring="f1_macro")
print(scores.mean(), scores.std())

In [ ]:
from sklearn.inspection import permutation_importance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

best_pipe = balanced_subsample_pipe
best_pipe.fit(X, y)

result = permutation_importance(
            best_pipe,                  
            X, y,
            n_repeats=20,
            scoring="f1_macro",
            random_state=42,
            n_jobs=-1
         )

feature_names = best_pipe.named_steps["prep"].get_feature_names_out()
importances   = pd.Series(result.importances_mean, index=X.columns)

print(importances)

topk = importances.sort_values(ascending=False).head(25)
plt.figure(figsize=(8, 10))
topk[::-1].plot(kind="barh")
plt.xlabel("Decrease in macro-F1 when permuted")
plt.title("Permutation feature importance (20 repeats)")
plt.tight_layout()
plt.show()

In [ ]:
balanced_pipe.fit(X, y)
balanced_pipe.named_steps["model"].feature_importances_

ohe = balanced_pipe.named_steps['prep'].named_transformers_['cat']
num_features = numerical_cols
cat_features = ohe.get_feature_names_out(categorical_cols)
all_features = np.concatenate([num_features, cat_features])

importances = balanced_pipe.named_steps["model"].feature_importances_

feat_importances = pd.DataFrame({
    'feature': all_features,
    'importance': importances
}).sort_values(by='importance', ascending=False)

feat_importances.head(50)

In [ ]:
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import cross_val_score

import xgboost as xgb

label_encoder = LabelEncoder()
xgb_pipe = Pipeline([
        ("prep",  preprocess),
        ("model", xgb.XGBClassifier(objective="multi:softprob", random_state=42))
])

X_t = preprocess.fit_transform(X)
y_t = label_encoder.fit_transform(y)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(xgb_pipe, X, y_t,
                         cv=cv,
                         scoring="f1_macro")
predictions = cross_val_predict(xgb_pipe, X, y_t, cv=cv)
print("XGB f1:", scores.mean())
print(classification_report(y, label_encoder.inverse_transform(predictions)))

XGB f1: 0.7500964182527807
              precision    recall  f1-score   support

   Artificer       0.81      0.65      0.72       204
   Barbarian       0.83      0.81      0.82       882
        Bard       0.66      0.64      0.65       661
      Cleric       0.84      0.84      0.84       969
       Druid       0.76      0.78      0.77       706
     Fighter       0.72      0.73      0.73      1342
        Monk       0.75      0.74      0.74       692
     Paladin       0.83      0.80      0.81       838
      Ranger       0.67      0.65      0.66       709
       Rogue       0.72      0.79      0.76      1187
    Sorcerer       0.78      0.73      0.75       617
     Warlock       0.64      0.64      0.64       623
      Wizard       0.85      0.87      0.86       719

    accuracy                           0.76     10149
   macro avg       0.76      0.74      0.75     10149
weighted avg       0.76      0.76      0.76     10149



In [76]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

label_encoder = LabelEncoder()

xgb_pipe = Pipeline([
        ("prep",  preprocess),
        ("model", xgb.XGBClassifier(objective="multi:softprob", random_state=42))
])

y_t = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_t, random_state=42)

xgb_pipe.fit(X_train, y_train)

y_pred = xgb_pipe.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')

print("F1 Macro Score:", f1)

print(classification_report(label_encoder.inverse_transform(y_test), label_encoder.inverse_transform(y_pred)))


F1 Macro Score: 0.7823845165291362
              precision    recall  f1-score   support

   Artificer       0.93      0.68      0.79        57
   Barbarian       0.87      0.85      0.86       204
        Bard       0.68      0.64      0.66       168
      Cleric       0.86      0.86      0.86       236
       Druid       0.80      0.82      0.81       167
     Fighter       0.75      0.76      0.75       353
        Monk       0.80      0.76      0.78       184
     Paladin       0.82      0.80      0.81       221
      Ranger       0.70      0.66      0.68       168
       Rogue       0.76      0.84      0.80       313
    Sorcerer       0.83      0.83      0.83       145
     Warlock       0.61      0.65      0.63       139
      Wizard       0.92      0.90      0.91       183

    accuracy                           0.79      2538
   macro avg       0.79      0.77      0.78      2538
weighted avg       0.79      0.79      0.79      2538

